STRONTIUM ISOTOPIC ANALYSIS

Load packages

In [ ]:
library(tidyverse)
library(scales)
options(scipen = 999)
library(readxl)

Load dataset

In [ ]:
root <- dirname(getwd())
DATA_DIR <- file.path(root, 'Dataset')
OUTPUT_DIR <- file.path(root, 'Results','Figures')

#Read in 2024 SWPA dataset samples
dataset_2024 <- read_csv(file.path(DATA_DIR, "2024_swpa_dataset.csv"))

#USGS PW database data
sr_pw_2023 <- read_excel(file.path(DATA_DIR, 'USGS_produced_water_data_2023.xlsx'))


# sr_uog <- read_csv(file.path(DATA_DIR, "Sr_UOG_2023.csv"))
# sr_cog <- read_csv(file.path(DATA_DIR, "Sr_COG_2023.csv"))

Regroup the samples into spill, impoundment, hotspot and control

In [ ]:
sr_uog <- sr_pw_2023 %>%
    filter(BASIN == 'Appalachian',
        WELLTYPE == 'Shale Gas',
        STATE == 'Pennsylvania',
        COUNTY %in% c('Greene','Washington','Westmoreland'))

sr_cog <- sr_pw_2023 %>%
    filter(BASIN == 'Appalachian',
        WELLTYPE == 'Conventional Hydrocarbon',
        STATE == 'Pennsylvania',
        COUNTY %in% c('Greene','Washington','Westmoreland'))

dim(sr_uog)
dim(sr_cog)

In [ ]:
# display min and max Sr ratios in UOG PW dataset
min_Sr87Sr86 <- min(sr_uog$Sr87Sr86, na.rm = TRUE)
max_Sr87Sr86 <- max(sr_uog$Sr87Sr86, na.rm = TRUE)
print(paste("Min 87Sr/86Sr in UOG PW:", round(min_Sr87Sr86, 6)))
print(paste("Max 87Sr/86Sr in UOG PW:", round(max_Sr87Sr86, 6)))

In [ ]:
#what are the current groups?
unique(dataset_2024$`Sample Group`)

# Remove stream samples & samples with Site ID = "Sample_022", "Sample_029", "Sample_091" (impacted by AMD or rainwater dilution)
dataset_2024 <- dataset_2024 %>%
  filter(`Sample Group` != "stream") %>%
  filter(!`Site ID` %in% c("Sample_022", "Sample_029", "Sample_091"))

#Create new groupings

#Control
control <- dataset_2024 %>%
  filter(`Sample Group` %in% c("C"))


#Hotspot
hotspot <- dataset_2024 %>%
  filter(`Sample Group` %in% c("HS", "HS/S", "HS/I"))

#Spill
spill <- dataset_2024 %>%
  filter(`Sample Group` %in% c("S", "HS/S"))

#Impoundment
impoundment <- dataset_2024 %>%
  filter(`Sample Group` %in% c("I", "HS/I"))

#Extra
extra <- dataset_2024 %>%
  filter(`Sample Group` %in% c("Extra"))

# **Final Figures**

In [ ]:
# Check what font families are available
# Load the systemfonts package
library(systemfonts)
# find Seogoe UI font
system_fonts() %>% filter(family == "Segoe UI")

In [ ]:
# Prepare the sr_uog dataframe
sr_uog_long <- sr_uog %>%
  mutate(dataset = "uog produced waters")%>%
  # filter(COUNTY %in% c("Washington", "Greene", "Westmoreland"))%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

# Prepare the sr_cog dataframe
sr_cog_long <- sr_cog %>%
  mutate(dataset = "cog produced waters")%>%
  # filter(COUNTY %in% c("Washington", "Greene", "Westmoreland"))%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

# Prepare 2024 SWPA samples
control_long <- control %>%
  mutate(dataset = "control",
        Sr87Sr86 = `87Sr/86Sr`)%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

spill_long <- spill %>%
  mutate(dataset = "spill",
        Sr87Sr86 = `87Sr/86Sr`)%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

impoundment_long <- impoundment %>%
  mutate(dataset = "impoundment",
        Sr87Sr86 = `87Sr/86Sr`)%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

hotspot_long <- hotspot %>%
  mutate(dataset = "hotspot",
        Sr87Sr86 = `87Sr/86Sr`)%>%
  select(dataset, Sr, Ca, Cl, Sr87Sr86)

# Combine all datasets into one long dataframe
combined_data <- bind_rows(sr_uog_long, sr_cog_long, control_long, spill_long, impoundment_long, hotspot_long)

# Specify the desired order for the datasets
desired_order <- c("uog produced waters", "cog produced waters", "control", "spill", "impoundment", "hotspot")

# Convert the dataset column to a factor with the specified order
combined_data$dataset <- factor(combined_data$dataset, levels = desired_order)

# # Define custom colors
# custom_colors <- c(
#   "uog produced waters" = "#808080",
#   "cog produced waters" = "#bfbfbf",
#   "control" = "#91bfdb", 
#   "spill" = "#fc8d59",  
#   "impoundment" = "#fee090", 
#   "hotspot" = "#d73027"
# )


# # Plotting
# ggplot(combined_data, aes(x = dataset, y = Sr87Sr86, fill = dataset)) +
#   geom_boxplot(aes(group = interaction(dataset))) +
#   theme_classic() +
#   scale_x_discrete(labels = c("uog produced waters" = "UOG PW",
#                               "cog produced waters" = "COG PW",
#                               "control" = "Control",
#                               "spill" = "Spill",
#                               "impoundment" = "Impoundment",
#                               "hotspot" = "Hotspot")) +
#   scale_fill_manual(values = custom_colors) +
#   #scale_color_manual(values = custom_colors) +
#   labs(y = "87Sr/86Sr", y = NULL) + 
#   theme(text = element_text(size =12),
#         axis.text.x = element_text(size = 11),
#         panel.background = element_rect(fill = "white", color = NA), # Set background color
#         panel.border = element_rect(color = "black", fill = NA, size = 1), # Add a black border
#         legend.position = "none")


In [ ]:
# New color palette
#install.packages("viridis")
library(viridis)

In [ ]:
# print min and max sr ratios
print(paste("Min 87Sr/86Sr in UOG PW:", round(min_Sr87Sr86, 6)))
print(paste("Max 87Sr/86Sr in UOG PW:", round(max_Sr87Sr86, 6)))

In [ ]:
# Generate the rocket color palette
rocket_palette <- viridis_pal(option = "viridis")(5)

custom_colors <- c(
  "UOG" = "#4d4d4d", 
  "COG" = "gray",
  "control" = "#91bfdb", 
  "spill" = "#fc8d59",  
  "impoundment" = "#fee090", 
  "hotspot" = "#d73027"
)

In [ ]:
ggplot(
  combined_data %>%
    filter(!dataset %in% c("uog produced waters","cog produced waters")) %>%
    mutate(SrCl = Sr / Cl),
  aes(SrCl, Sr87Sr86,
      color = dataset,
      shape = dataset,      # <-- map shape to dataset
      size  = Cl)
) +
  geom_hline(yintercept = min_Sr87Sr86, linetype = "dashed", color = "grey") +
  geom_hline(yintercept = max_Sr87Sr86, linetype = "dashed", color = "grey") +
  geom_point(show.legend = TRUE) +
  # UOG overlay: fixed square (shape 15), excluded from legend
  geom_point(
    data = sr_uog_long %>% mutate(SrCl = Sr / Cl, dataset = "UOG"),
    aes(SrCl, Sr87Sr86, color = dataset, shape = dataset),  # keep color mapping if you want it colored
    size = 2
  ) +
  theme_classic(base_size = 12) +
  theme(
    panel.background = element_rect(fill = "white", color = NA),
    panel.border     = element_rect(color = "black", fill = NA, size = 1)
  ) +
  labs(
    y = expression(""^"87"*"Sr"/""^"86"*"Sr"),
    x = expression("[Sr] / [Cl] Mass Ratio"),
    size = "[Cl] (mg/L)"
  ) +
  scale_x_log10() +
  annotate("text", x = 0.5, y = min_Sr87Sr86, label = "UOG Min.",  color = "grey", vjust = -1,  hjust = -0.1) +
  annotate("text", x = 0.5, y = max_Sr87Sr86, label = "UOG Max.",  color = "grey", vjust =  1.5, hjust = -0.1) +
  scale_color_manual(
    name = "Dataset",  # Legend title
    limits = c("control", "spill", "impoundment", "hotspot","UOG"),  # Ensure all levels are shown
    values = custom_colors,
    drop = FALSE  # Keep all levels in the legend, even if not present in data
    ) +
  # Set shapes per dataset (names must match levels in `dataset`)
  scale_shape_manual(
    name = "Dataset",  # Legend title
    limits = c("control", "spill", "impoundment", "hotspot","UOG"),  # Ensure all levels are shown
    values = c(
    "UOG" = 15,          # square
    "control" = 3,      # plus
    "spill" = 18,       # diamond
    "impoundment" = 18,  # diamond
    "hotspot" = 18        # diamond
    ),
    drop = FALSE  # Keep all levels in the legend, even if not present in data
  ) 
  
  # Optional: tidy legends (separate orders; make color legend points a bit larger)
  # guides(
  #   color = guide_legend(order = 1, override.aes = list(size = 3)),
  #   shape = guide_legend(order = 2),
  #   size  = guide_legend(order = 3)
  # )


ggsave(file.path(OUTPUT_DIR, "Figure 2B.pdf"), width = 9, height = 6)